In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [4]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [5]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [6]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [7]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [8]:
device = provider.get_backend('ibm_cairo')
device

<IBMQBackend('ibm_cairo') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [9]:
device.configuration().to_dict()['max_shots']

100000

In [10]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [11]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [12]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [13]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [14]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [15]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68528/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [16]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [17]:
#checking circuit

In [18]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [19]:
circuit = front_layer.copy()

In [20]:
apply_checking_circuit(circuit, [1], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [1], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68528/3853186367.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68528/3853186367.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [21]:
circuit.draw()

┌───┐      ┌───────────────┐      ┌────────────────┐      »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├──────»
           ├───┤      ├───────────────┤    │ └────────────────┘      »
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■─────────────■─»
     ┌─────┴───┴─────┐└───────────────┘ │            │             │ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■──────────■──┼─»
     ├───────────────┤                  │                       │  │ »
q_3: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────■──┼─»
     └─────┬───┬─────┘                  │                          │ »
q_4: ──────┤ H ├────────────────────────■──────────────────────────■─»
           └───┘                                                     »
«                      
«q_0: ─────────────────
«     ┌───────────────┐
«q_1: ┤ Ry(-0.015751) ├
«     ├───────────────┤
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └────┬───┬─────┘
«q_4: ──────┤ H ├──────
«           └───┘

In [22]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [23]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [24]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_68528/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [25]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐      ┌────────────────┐┌────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π) ├»
           ├───┤      ├───────────────┤    │ └────────────────┘└────────────┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■───────────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │            │                       »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■───────────────■───────»
     ├───────────────┤                  │                            │       »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────────────────────────■───────»
     └─────┬───┬─────┘                  │                                    »
q_4: ──────┤ H ├────────────────────────■────────────────────────────────────»
           └───┘                                                             »
«                                       
«q_0: ──────────────────────────────────
«        ┌───────────────┐┌────────────┐
«q_1: ─■─┤ Ry(-0.015751) ├┤ U(π/2,0,π) ├
«      │ ├───────────────┤├────────────┤
«q_2: ─┼─┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├
«      │ └┬──────────────┤├────────────┤
«q_3: ─┼──┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«      │  └────┬───┬─────┘└────────────┘
«q_4: ─■───────┤ H ├────────────────────
«              └───┘

In [26]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [29]:
cuts = [(circuit.qubits[1], 3),(circuit.qubits[1], 5)]

In [30]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐      ┌────────────────┐┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤    │ └────────────────┘└──────────────┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■─────────────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │            │                         »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■────────────────■────────»
     ├───────────────┤                  │                             │        »
q_3: ┤ Ry(-0.019851) ├──────────────────┼─────────────────────────────■────────»
     └─────┬───┬─────┘                  │                                      »
q_4: ──────┤ H ├────────────────────────■──────────────────────────────────────»
           └───┘                                                               »
«                                         
«q_0: ────────────────────────────────────
«        ┌─────────────

           ┌───┐      ┌───────────────┐      ┌────────────────┐┌────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├┤ U(π/2,0,π) ├»
           ├───┤      ├───────────────┤    │ └────────────────┘└────────────┘»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■───────────────────────»
     ┌─────┴───┴─────┐└───────────────┘ │            │                       »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■───────────────■───────»
     ├───────────────┤                  │                            │       »
q_3: ┤ Ry(-0.019851) ├──────────────────┼────────────────────────────■───────»
     └─────┬───┬─────┘                  │                                    »
q_4: ──────┤ H ├────────────────────────■────────────────────────────────────»
           └───┘                                                             »
«                         
«q_0: ────────────────────
«        ┌───────────────┐
«q_1: ─■─┤ Ry(-0.015751) ├
«      │ ├─────────────

           ┌───┐      ┌───────────────┐      ┌────────────────┐      »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├──────»
           ├───┤      ├───────────────┤    │ └────────────────┘      »
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■─────────────■─»
     ┌─────┴───┴─────┐└───────────────┘ │            │             │ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■──────────■──┼─»
     ├───────────────┤                  │                       │  │ »
q_3: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────■──┼─»
     └─────┬───┬─────┘                  │                          │ »
q_4: ──────┤ H ├────────────────────────■──────────────────────────■─»
           └───┘                                                     »
«                                    
«q_0: ───────────────────────────────
«     ┌───────────────┐              
«q_1: ┤ Ry(-0.015751) ├──────────────
«     ├───────────────┤              
«q_2: ┤ Ry(0.0036205) ├──────

In [31]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 1)),), Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 0)), (0, Qubit(QuantumRegister(4, 'q'), 3)), (1, Qubit(QuantumRegister(3, 'q'), 1))), Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 0)),), Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 2)),)}


In [32]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [33]:
total_variants

24

In [34]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [35]:
fragments_list[0][0].draw()

┌───────────────┐                                                         »
q_0: ┤ Ry(0.0038243) ├───────────────────────────────────────■─────────────────»
     └─────┬───┬─────┘┌───────────────┐   ┌────────────────┐ │ ┌──────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼─┤ U(π/2,0,π/2) ├»
     ┌─────┴───┴─────┐└───────────────┘ │ └────────────────┘ │ └──────────────┘»
q_2: ┤ Ry(-0.019851) ├──────────────────┼────────────────────┼─────────────────»
     └───────────────┘                  │                    │                 »
q_3: ───────────────────────────────────■────────────────────■─────────────────»
                                                                               »
«        ┌───────────────┐┌──────────────┐
«q_0: ─■─┤ Ry(0.0036205) ├┤ U(π/2,0,π/2) ├
«      │ └───────────────┘└──────────────┘
«q_1: ─┼──────────────────────────────────
«      │  ┌──────────────┐┌──────────────┐
«q_2: ─■──┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├
«         └──────────────┘└──────────────┘
«q_3: ────────────────────────────────────
«

In [36]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
     ┌───────────────┐                                                         »
q_0: ┤ Ry(0.0038243) ├───────────────────────────────────────■─────────────────»
     └─────┬───┬─────┘┌───────────────┐   ┌────────────────┐ │ ┌──────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼─┤ U(π/2,0,π/2) ├»
     ┌─────┴───┴─────┐└───────────────┘ │ └────────────────┘ │ └──────────────┘»
q_2: ┤ Ry(-0.019851) ├──────────────────┼────────────────────┼─────────────────»
     └───────────────┘                  │                    │                 »
q_3: ───────────────────────────────────■────────────────────■─────────────────»
                                                                               »
«        ┌───────────────┐┌──────────────┐
«q_0: ─■─┤ Ry(0.0036205) ├┤ U(π/2,0,π/2) ├
«      │ └───────────────┘└──────────────┘
«q_1: ─┼──────────────────────────────────
«      │  ┌──────────────┐┌──────────────┐
«q_2: ─■──┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├
«

In [37]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [38]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 1)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((1, Qubit(QuantumRegister(3, 'q'), 0)),
  (0, Qubit(QuantumRegister(4, 'q'), 3)),
  (1, Qubit(QuantumRegister(3, 'q'), 1))),
 Qubit(QuantumRegister(5, 'q'), 2): ((0, Qubit(QuantumRegister(4, 'q'), 0)),),
 Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 2)),)}

In [39]:
num_fragments = len(fragments)
num_fragments

2

In [40]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [41]:
fragments[0].draw()

┌───────────────┐                                            »
q_0: ┤ Ry(0.0038243) ├───────────────────────────────────────■──■─»
     └─────┬───┬─────┘┌───────────────┐   ┌────────────────┐ │  │ »
q_1: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┼──┼─»
     ┌─────┴───┴─────┐└───────────────┘ │ └────────────────┘ │  │ »
q_2: ┤ Ry(-0.019851) ├──────────────────┼────────────────────┼──■─»
     └───────────────┘                  │                    │    »
q_3: ───────────────────────────────────■────────────────────■────»
                                                                  »
«     ┌───────────────┐
«q_0: ┤ Ry(0.0036205) ├
«     └───────────────┘
«q_1: ─────────────────
«      ┌──────────────┐
«q_2: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_3: ─────────────────
«

In [42]:
fragments[1].draw()

┌───┐┌───────────────┐                       
q_0: ┤ X ├┤ Ry(-0.015869) ├─■─────────────────────
     └───┘└───────────────┘ │    ┌───────────────┐
q_1: ───────────────────────┼──■─┤ Ry(-0.015751) ├
     ┌───┐                  │  │ └─────┬───┬─────┘
q_2: ┤ H ├──────────────────■──■───────┤ H ├──────
     └───┘                             └───┘

In [53]:
hardware_index = 0

In [44]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, initial_layout = [23,25,24,26], opt_lvl = 3, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [45]:
#contains 25 basis
len(tomography_circuits)

25

In [46]:
#contains three fragments
len(tomography_circuits[0])

2

In [47]:
len(tomography_circuits[0][1])

1

In [48]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [49]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [51]:
multi_exp_circuits[0][-200].draw()

░ ┌───────────────┐                    »
  q_0: ────────────────────░─┤ Ry(0.0038243) ├────────────────────»
                           ░ └─────┬───┬─────┘┌───────────────┐   »
  q_1: ────────────────────░───────┤ X ├──────┤ Ry(-0.003534) ├─■─»
                           ░ ┌─────┴───┴─────┐└───────────────┘ │ »
  q_2: ────────────────────░─┤ Ry(-0.019851) ├──────────────────┼─»
       ┌─────────────────┐ ░ └───────────────┘                  │ »
  q_3: ┤ U3(-1.9106,π,0) ├─░────────────────────────────────────■─»
       └─────────────────┘ ░                                      »
c16_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_3: ═══════════════════════════════════════════════════════════»
                                                                  »
«                                               ┌───────────────┐┌────────────┐»
«  q_0: ───────────────────■──────────────────■─┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├»
«       ┌────────────────┐ │ ┌──────────────┐ │ └───────────────┘└────────────┘»
«  q_1: ┤ Ry(-0.0036834) ├─┼─┤ U(π/2,0,π/2) ├─┼────────────────────────────────»
«       └────────────────┘ │ └──────────────┘ │  ┌──────────────┐┌────────────┐»
«  q_2: ───────────────────┼──────────────────■──┤ Ry(0.019873) ├┤ U(π/2,0,π) ├»
«                          │                     └──────────────┘└────────────┘»
«  q_3: ───────────────────■───────────────────────────────────────────────────»
«                                                                              »
«c16_0: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_1: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_2: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_3: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«        ░ ┌─┐              
«  q_0: ─░─┤M├──────────────
«        ░ └╥┘┌─┐           
«  q_1: ─░──╫─┤M├───────────
«        ░  ║ └╥┘┌─┐        
«  q_2: ─░──╫──╫─┤M├────────
«        ░  ║  ║ └╥┘┌───┐┌─┐
«  q_3: ─░──╫──╫──╫─┤ H ├┤M├
«        ░  ║  ║  ║ └───┘└╥┘
«c16_0: ════╩══╬══╬═══════╬═
«              ║  ║       ║ 
«c16_1: ═══════╩══╬═══════╬═
«                 ║       ║ 
«c16_2: ══════════╩═══════╬═
«                         ║ 
«c16_3: ══════════════════╩═
«

In [54]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [25,23,26,24], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  6377128309560769183d0aa5
Job Status: job is queued (125)   

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/6377128309560769183d0aa5/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7f8511d24310>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [55]:
frag_datas[0] = [device.retrieve_job(job_id='6377128309560769183d0aa5').result()]

In [56]:
frag_datas[1] = run_circuits(multi_exp_circuits[1], initial_layout = None,
                             backend = simulation_backend, shots = shots // total_variants,
                             optimization_level = 3, monitor_jobs = True, noise_model = None)

job_id:  be743f5f-7cc4-4e30-a758-8fc712eefaad
Job Status: job has successfully run


In [57]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [58]:
num_tomo_circuits

25

In [59]:
test = [[], []]

In [60]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [61]:
frag_datas[0][0].get_counts()

[{'0000': 3986,
  '0001': 4007,
  '0010': 4034,
  '0011': 3720,
  '0100': 3987,
  '0101': 4094,
  '0110': 3563,
  '0111': 3790,
  '1000': 83,
  '1001': 106,
  '1010': 102,
  '1011': 110,
  '1100': 83,
  '1101': 111,
  '1110': 106,
  '1111': 118},
 {'0000': 2124,
  '0001': 2062,
  '0010': 2023,
  '0011': 1999,
  '0100': 1980,
  '0101': 2067,
  '0110': 2003,
  '0111': 2029,
  '1000': 2023,
  '1001': 2176,
  '1010': 1963,
  '1011': 1969,
  '1100': 2029,
  '1101': 1985,
  '1110': 1859,
  '1111': 1709},
 {'0000': 2150,
  '0001': 2150,
  '0010': 2093,
  '0011': 2164,
  '0100': 2100,
  '0101': 2094,
  '0110': 2036,
  '0111': 2016,
  '1000': 2079,
  '1001': 2046,
  '1010': 1811,
  '1011': 1782,
  '1100': 2018,
  '1101': 1968,
  '1110': 1657,
  '1111': 1836},
 {'0000': 1549,
  '0001': 1515,
  '0010': 1232,
  '0011': 1215,
  '0100': 1526,
  '0101': 1563,
  '0110': 1203,
  '0111': 1191,
  '1000': 2376,
  '1001': 2644,
  '1010': 2725,
  '1011': 2845,
  '1100': 2231,
  '1101': 2580,
  '1110': 2596,

In [62]:
frag_datas[0][0].get_counts()

[{'0000': 3986,
  '0001': 4007,
  '0010': 4034,
  '0011': 3720,
  '0100': 3987,
  '0101': 4094,
  '0110': 3563,
  '0111': 3790,
  '1000': 83,
  '1001': 106,
  '1010': 102,
  '1011': 110,
  '1100': 83,
  '1101': 111,
  '1110': 106,
  '1111': 118},
 {'0000': 2124,
  '0001': 2062,
  '0010': 2023,
  '0011': 1999,
  '0100': 1980,
  '0101': 2067,
  '0110': 2003,
  '0111': 2029,
  '1000': 2023,
  '1001': 2176,
  '1010': 1963,
  '1011': 1969,
  '1100': 2029,
  '1101': 1985,
  '1110': 1859,
  '1111': 1709},
 {'0000': 2150,
  '0001': 2150,
  '0010': 2093,
  '0011': 2164,
  '0100': 2100,
  '0101': 2094,
  '0110': 2036,
  '0111': 2016,
  '1000': 2079,
  '1001': 2046,
  '1010': 1811,
  '1011': 1782,
  '1100': 2018,
  '1101': 1968,
  '1110': 1657,
  '1111': 1836},
 {'0000': 1549,
  '0001': 1515,
  '0010': 1232,
  '0011': 1215,
  '0100': 1526,
  '0101': 1563,
  '0110': 1203,
  '0111': 1191,
  '1000': 2376,
  '1001': 2644,
  '1010': 2725,
  '1011': 2845,
  '1100': 2231,
  '1101': 2580,
  '1110': 2596,

In [63]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'X')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'X')) ('S0',) ('X',) [3] [3]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Y')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Y')) ('S0',) ('Y',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'X')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'X')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'X')) ('S1',) ('X',) [3] [3]
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Y')) ('S0', 'S0', 'S0', 'S1') ('Z', 'Z', 'Z', 'Y')
(('S0', 'S0', 'S0', 'S1'), ('Z', 'Z', 'Z', 'Y')) ('S1',) ('Y',) [3] [3

In [64]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1


In [65]:
len(organized_data)

25

In [66]:
organized_data[2]

[{'000': {(('S0',), ('Zp',)): 142,
   (('S0',), ('Zm',)): 5,
   (('S0',), ('Xp',)): 75,
   (('S0',), ('Xm',)): 62,
   (('S0',), ('Yp',)): 74,
   (('S0',), ('Ym',)): 69,
   (('S1',), ('Zp',)): 63,
   (('S1',), ('Zm',)): 93,
   (('S1',), ('Xp',)): 73,
   (('S1',), ('Xm',)): 95,
   (('S1',), ('Yp',)): 68,
   (('S1',), ('Ym',)): 73,
   (('S2',), ('Zp',)): 60,
   (('S2',), ('Zm',)): 84,
   (('S2',), ('Xp',)): 86,
   (('S2',), ('Xm',)): 70,
   (('S2',), ('Yp',)): 86,
   (('S2',), ('Ym',)): 74,
   (('S3',), ('Zp',)): 61,
   (('S3',), ('Zm',)): 111,
   (('S3',), ('Xp',)): 106,
   (('S3',), ('Xm',)): 59,
   (('S3',), ('Yp',)): 63,
   (('S3',), ('Ym',)): 73},
  '001': {(('S0',), ('Zp',)): 147,
   (('S0',), ('Zm',)): 2,
   (('S0',), ('Xp',)): 68,
   (('S0',), ('Xm',)): 65,
   (('S0',), ('Yp',)): 81,
   (('S0',), ('Ym',)): 65,
   (('S1',), ('Zp',)): 53,
   (('S1',), ('Zm',)): 122,
   (('S1',), ('Xp',)): 56,
   (('S1',), ('Xm',)): 100,
   (('S1',), ('Yp',)): 77,
   (('S1',), ('Ym',)): 81,
   (('S2'

In [67]:
organized_data[-1][-1]

{'11': {(('S0',), ('Zm',)): 2,
  (('S0',), ('Xm',)): 4,
  (('S0',), ('Xp',)): 4,
  (('S0',), ('Ym',)): 4,
  (('S0',), ('Yp',)): 4,
  (('S1',), ('Zm',)): 2,
  (('S1',), ('Zp',)): 5,
  (('S1',), ('Xm',)): 11,
  (('S1',), ('Ym',)): 4,
  (('S2',), ('Zp',)): 6,
  (('S2',), ('Zm',)): 1,
  (('S2',), ('Xp',)): 4,
  (('S2',), ('Xm',)): 3,
  (('S2',), ('Yp',)): 1,
  (('S2',), ('Ym',)): 6,
  (('S3',), ('Zp',)): 1,
  (('S3',), ('Zm',)): 1,
  (('S3',), ('Xp',)): 2,
  (('S3',), ('Xm',)): 2,
  (('S3',), ('Yp',)): 6,
  (('S3',), ('Ym',)): 2,
  (('S0',), ('Zp',)): 0,
  (('S1',), ('Xp',)): 0,
  (('S1',), ('Yp',)): 0},
 '00': {(('S0',), ('Zp',)): 5,
  (('S0',), ('Xp',)): 3,
  (('S0',), ('Xm',)): 2,
  (('S0',), ('Yp',)): 1,
  (('S0',), ('Ym',)): 5,
  (('S1',), ('Zm',)): 3,
  (('S1',), ('Zp',)): 2,
  (('S1',), ('Xp',)): 7,
  (('S1',), ('Xm',)): 1,
  (('S1',), ('Ym',)): 2,
  (('S1',), ('Yp',)): 2,
  (('S2',), ('Zm',)): 5,
  (('S2',), ('Zp',)): 3,
  (('S2',), ('Xm',)): 7,
  (('S2',), ('Ym',)): 6,
  (('S3',),

In [85]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'000': {(('S0',), ('Zp',)): 3986, (('S0',), ('Zm',)): 83, (('S0',), ('Xp',)): 2124, (('S0',), ('Xm',)): 2023, (('S0',), ('Yp',)): 2150, (('S0',), ('Ym',)): 2079, (('S1',), ('Zp',)): 1549, (('S1',), ('Zm',)): 2376, (('S1',), ('Xp',)): 231, (('S1',), ('Xm',)): 3620, (('S1',), ('Yp',)): 1433, (('S1',), ('Ym',)): 2404, (('S2',), ('Zp',)): 1481, (('S2',), ('Zm',)): 2375, (('S2',), ('Xp',)): 2417, (('S2',), ('Xm',)): 1505, (('S2',), ('Yp',)): 3638, (('S2',), ('Ym',)): 267, (('S3',), ('Zp',)): 1500, (('S3',), ('Zm',)): 2167, (('S3',), ('Xp',)): 2952, (('S3',), ('Xm',)): 617, (('S3',), ('Yp',)): 752, (('S3',), ('Ym',)): 2917}, '001': {(('S0',), ('Zp',)): 4007, (('S0',), ('Zm',)): 106, (('S0',), ('Xp',)): 2062, (('S0',), ('Xm',)): 2176, (('S0',), ('Yp',)): 2150, (('S0',), ('Ym',)): 2046, (('S1',), ('Zp',)): 1515, (('S1',), ('Zm',)): 2644, (('S1',), ('Xp',)): 256, (('S1',), ('Xm',)): 3797, (('S1',), ('Yp',)): 1824, (('S1',), ('Ym',)): 2394, (('S2',), ('Zp',)): 1477, (('S2',), ('Zm',)): 2678, ((

{'000': {(('S0',), ('Zp',)): 4250, (('S0',), ('Zm',)): 124, (('S0',), ('Xp',)): 2130, (('S0',), ('Xm',)): 2206, (('S0',), ('Yp',)): 2272, (('S0',), ('Ym',)): 1996, (('S1',), ('Zp',)): 1315, (('S1',), ('Zm',)): 2997, (('S1',), ('Xp',)): 391, (('S1',), ('Xm',)): 3933, (('S1',), ('Yp',)): 1715, (('S1',), ('Ym',)): 2639, (('S2',), ('Zp',)): 1448, (('S2',), ('Zm',)): 3082, (('S2',), ('Xp',)): 2708, (('S2',), ('Xm',)): 1733, (('S2',), ('Yp',)): 4104, (('S2',), ('Ym',)): 424, (('S3',), ('Zp',)): 1360, (('S3',), ('Zm',)): 2883, (('S3',), ('Xp',)): 3430, (('S3',), ('Xm',)): 874, (('S3',), ('Yp',)): 862, (('S3',), ('Ym',)): 3486}, '001': {(('S0',), ('Zp',)): 4032, (('S0',), ('Zm',)): 176, (('S0',), ('Xp',)): 2112, (('S0',), ('Xm',)): 2148, (('S0',), ('Yp',)): 2325, (('S0',), ('Ym',)): 2007, (('S1',), ('Zp',)): 1316, (('S1',), ('Zm',)): 3308, (('S1',), ('Xp',)): 462, (('S1',), ('Xm',)): 4159, (('S1',), ('Yp',)): 2014, (('S1',), ('Ym',)): 2574, (('S2',), ('Zp',)): 1362, (('S2',), ('Zm',)): 3304, (

{'11': {(('S0',), ('Zm',)): 3, (('S0',), ('Xp',)): 6, (('S0',), ('Xm',)): 3, (('S0',), ('Ym',)): 1, (('S0',), ('Yp',)): 2, (('S1',), ('Zp',)): 1, (('S1',), ('Xm',)): 5, (('S1',), ('Ym',)): 4, (('S1',), ('Yp',)): 7, (('S2',), ('Zm',)): 2, (('S2',), ('Zp',)): 7, (('S2',), ('Xm',)): 1, (('S2',), ('Xp',)): 4, (('S2',), ('Ym',)): 5, (('S3',), ('Zm',)): 2, (('S3',), ('Zp',)): 6, (('S3',), ('Xp',)): 9, (('S3',), ('Xm',)): 1, (('S3',), ('Yp',)): 4, (('S0',), ('Zp',)): 0, (('S1',), ('Zm',)): 0, (('S1',), ('Xp',)): 0, (('S2',), ('Yp',)): 0, (('S3',), ('Ym',)): 0}, '00': {(('S0',), ('Zp',)): 5, (('S0',), ('Xp',)): 4, (('S0',), ('Xm',)): 2, (('S0',), ('Ym',)): 2, (('S0',), ('Yp',)): 2, (('S1',), ('Zm',)): 4, (('S1',), ('Zp',)): 1, (('S1',), ('Xp',)): 4, (('S1',), ('Yp',)): 2, (('S1',), ('Ym',)): 2, (('S2',), ('Zm',)): 5, (('S2',), ('Xp',)): 1, (('S2',), ('Xm',)): 5, (('S2',), ('Yp',)): 1, (('S2',), ('Ym',)): 4, (('S3',), ('Zm',)): 6, (('S3',), ('Xm',)): 4, (('S3',), ('Xp',)): 3, (('S3',), ('Ym',))

{'000': {(('S0',), ('Zp',)): 7869, (('S0',), ('Zm',)): 220, (('S0',), ('Xp',)): 3905, (('S0',), ('Xm',)): 4271, (('S0',), ('Yp',)): 4409, (('S0',), ('Ym',)): 3862, (('S1',), ('Zp',)): 2467, (('S1',), ('Zm',)): 5974, (('S1',), ('Xp',)): 736, (('S1',), ('Xm',)): 7922, (('S1',), ('Yp',)): 3490, (('S1',), ('Ym',)): 5095, (('S2',), ('Zp',)): 2645, (('S2',), ('Zm',)): 6196, (('S2',), ('Xp',)): 5460, (('S2',), ('Xm',)): 3526, (('S2',), ('Yp',)): 7922, (('S2',), ('Ym',)): 909, (('S3',), ('Zp',)): 2678, (('S3',), ('Zm',)): 5907, (('S3',), ('Xp',)): 6756, (('S3',), ('Xm',)): 1893, (('S3',), ('Yp',)): 1593, (('S3',), ('Ym',)): 7027}, '001': {(('S0',), ('Zp',)): 302, (('S0',), ('Zm',)): 10, (('S0',), ('Xp',)): 167, (('S0',), ('Xm',)): 167, (('S0',), ('Yp',)): 158, (('S0',), ('Ym',)): 141, (('S1',), ('Zp',)): 104, (('S1',), ('Zm',)): 242, (('S1',), ('Xp',)): 101, (('S1',), ('Xm',)): 246, (('S1',), ('Yp',)): 173, (('S1',), ('Ym',)): 164, (('S2',), ('Zp',)): 98, (('S2',), ('Zm',)): 217, (('S2',), ('X

{'000': {(('S0',), ('Zp',)): 4119, (('S0',), ('Zm',)): 158, (('S0',), ('Xp',)): 2144, (('S0',), ('Xm',)): 2158, (('S0',), ('Yp',)): 2215, (('S0',), ('Ym',)): 2016, (('S1',), ('Zp',)): 1284, (('S1',), ('Zm',)): 3496, (('S1',), ('Xp',)): 449, (('S1',), ('Xm',)): 4370, (('S1',), ('Yp',)): 2048, (('S1',), ('Ym',)): 2829, (('S2',), ('Zp',)): 1478, (('S2',), ('Zm',)): 3469, (('S2',), ('Xp',)): 3096, (('S2',), ('Xm',)): 1883, (('S2',), ('Yp',)): 4379, (('S2',), ('Ym',)): 573, (('S3',), ('Zp',)): 1405, (('S3',), ('Zm',)): 3339, (('S3',), ('Xp',)): 3530, (('S3',), ('Xm',)): 1118, (('S3',), ('Yp',)): 878, (('S3',), ('Ym',)): 3919}, '001': {(('S0',), ('Zp',)): 4126, (('S0',), ('Zm',)): 110, (('S0',), ('Xp',)): 2083, (('S0',), ('Xm',)): 2188, (('S0',), ('Yp',)): 2263, (('S0',), ('Ym',)): 1882, (('S1',), ('Zp',)): 1324, (('S1',), ('Zm',)): 2893, (('S1',), ('Xp',)): 395, (('S1',), ('Xm',)): 3818, (('S1',), ('Yp',)): 1617, (('S1',), ('Ym',)): 2551, (('S2',), ('Zp',)): 1397, (('S2',), ('Zm',)): 2963, 

In [69]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [70]:
direct_recombined_dist

{'10010': 5.923834702483024e-07,
 '00000': 1.6077434824718857e-06,
 '10000': 0.0005860982452732572,
 '00010': 0.018191170417435225,
 '10110': -2.9056104969730446e-09,
 '00100': 8.798279112079822e-08,
 '10100': -2.4446481906996576e-05,
 '00110': 0.0005891734694746236,
 '10011': 4.363572236598605e-05,
 '00001': -1.769814169079284e-05,
 '10001': 0.009922176106666661,
 '00011': 0.9178466535743881,
 '10111': 1.4779980070398871e-06,
 '00101': 1.4139105087443248e-06,
 '10101': 0.0016809739814254267,
 '00111': 0.03507578958557308,
 '11010': 1.643086821335895e-08,
 '01000': 3.041717828659912e-08,
 '11000': 6.490699404010895e-05,
 '01010': 0.0003166002938409959,
 '11011': 7.184301895993732e-07,
 '01001': -3.440317100460636e-07,
 '11001': 7.827104428882347e-05,
 '01011': 0.014705423174569503,
 '11111': 4.986152215299465e-08,
 '01101': 2.6616223312880703e-08,
 '11101': -3.3479562373238507e-06,
 '01111': 0.0009388502642303884,
 '11110': 3.6662589770001726e-09,
 '01100': 4.519069021138048e-09,
 '111

In [71]:
likely_recombined_dist

{'10000': 0.00041000399362763724,
 '00010': 0.017801729428341777,
 '10100': 2.847985239497992e-06,
 '00110': 0.000259530376549445,
 '10001': 0.014781434257688098,
 '00011': 0.915466222783842,
 '10101': 0.0014274418843631827,
 '00111': 0.03463741675752124,
 '11001': 0.00017893036445283968,
 '01011': 0.0144108148132781,
 '11101': 3.586795701613745e-06,
 '01111': 0.0006200405593945731}

In [72]:
filtered_recombined_lists

[{'0010': 0.0527658116428895,
  '0000': 0.05237091173737207,
  '0110': 0.057892720864419045,
  '0100': 0.057733734925694113,
  '0011': 0.06310519613359589,
  '0001': 0.06309910037036515,
  '0111': 0.0677219408036574,
  '0101': 0.06794286212492313,
  '1010': 0.050432504017574126,
  '1000': 0.04999763955364833,
  '1110': 0.05894219693842989,
  '1100': 0.05874423498632061,
  '1011': 0.05909478199652643,
  '1001': 0.05908180147732339,
  '1111': 0.0698644436067725,
  '1101': 0.07003917118877276},
 {'0000': 0.20931760744234051,
  '0010': 0.20962802989289026,
  '0100': 0.007123655548017069,
  '0110': 0.007088440851288818,
  '0001': 0.24858827201827324,
  '0011': 0.24753705564320114,
  '0101': 0.007963952979621181,
  '0111': 0.007909116934158,
  '1000': 0.002558276971972134,
  '1010': 0.0025604731653296477,
  '1001': 0.002993506176650246,
  '1011': 0.002983832280516585},
 {'0010': 0.004155637721606105,
  '0110': 0.0050093736012982734,
  '0011': 0.23298689527235378,
  '0111': 0.2542444131077971

In [73]:
from utils.utils import filter_results

In [74]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [75]:
filter_direct_recombined

{'0000': 1.6077434824718857e-06,
 '0010': 0.018191170417435225,
 '0100': 8.798279112079822e-08,
 '0110': 0.0005891734694746236,
 '0001': -1.769814169079284e-05,
 '0011': 0.9178466535743881,
 '0101': 1.4139105087443248e-06,
 '0111': 0.03507578958557308,
 '1000': 3.041717828659912e-08,
 '1010': 0.0003166002938409959,
 '1001': -3.440317100460636e-07,
 '1011': 0.014705423174569503,
 '1101': 2.6616223312880703e-08,
 '1111': 0.0009388502642303884,
 '1100': 4.519069021138048e-09,
 '1110': -9.703772660773256e-08}

In [76]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [77]:
filter_likely_recombined

{'0010': 0.017801729428341777,
 '0110': 0.000259530376549445,
 '0011': 0.915466222783842,
 '0111': 0.03463741675752124,
 '1011': 0.0144108148132781,
 '1111': 0.0006200405593945731}

In [78]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [79]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [80]:
likely_recombined_dist['00011']

0.915466222783842

In [81]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

0.9311128718670603

In [82]:
norm_filter_dist

{'0010': 0.018105986872808334,
 '0110': 0.00026396612811213643,
 '0011': 0.9311128718670603,
 '0111': 0.0352294205820928,
 '1011': 0.014657116595664937,
 '1111': 0.0006306379542615379}

In [86]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = 1, Pauli = Pauli_tuple[0])  

In [87]:
final_expect_val

(-1.0543212782069218+0j)

In [88]:
import json
with open('LiH_cairo_check1.json', 'w') as f:
    json.dump(norm_filtered_lists, f)